In [9]:
# EMAL

import pandas as pd
import numpy as np

import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import save_npz

import spacy
from datetime import datetime

from email.parser import Parser
from email import policy



In [ ]:
#read data 
DATA_PATH = "emails.csv"
df = pd.read_csv(DATA_PATH, encoding='utf-8', on_bad_lines='skip')
print(df.shape[0])
df.head(10)

517401


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
5,allen-p/_sent_mail/1002.,Message-ID: <30965995.1075863688265.JavaMail.e...
6,allen-p/_sent_mail/1003.,Message-ID: <16254169.1075863688286.JavaMail.e...
7,allen-p/_sent_mail/1004.,Message-ID: <17189699.1075863688308.JavaMail.e...
8,allen-p/_sent_mail/101.,Message-ID: <20641191.1075855687472.JavaMail.e...
9,allen-p/_sent_mail/102.,Message-ID: <30795301.1075855687494.JavaMail.e...


In [3]:
#print(set([x for x in df['file'].tolist()[:20]]))
df['dir_owner']= df['file'].apply(lambda x: x.split('/')[0])
df['dir']= df['file'].apply(lambda x: x.split('/')[1])
df.tail(5)

,file,message,dir_owner,dir
517396,zufferli-j/sent_items/95.,Message-ID: <26807948.1075842029936.JavaMail.e...,zufferli-j,sent_items
517397,zufferli-j/sent_items/96.,Message-ID: <25835861.1075842029959.JavaMail.e...,zufferli-j,sent_items
517398,zufferli-j/sent_items/97.,Message-ID: <28979867.1075842029988.JavaMail.e...,zufferli-j,sent_items
517399,zufferli-j/sent_items/98.,Message-ID: <22052556.1075842030013.JavaMail.e...,zufferli-j,sent_items
517400,zufferli-j/sent_items/99.,Message-ID: <28618979.1075842030037.JavaMail.e...,zufferli-j,sent_items


In [4]:
print(f'Uniq Users {df.dir_owner.nunique()}')
print(f'Uniq files cnt {df.dir.nunique()}')
print(f'Uniq files {df.dir.unique().tolist()}', sep=',')     

Uniq Users 150
Uniq files cnt 1427
Uniq files ['_sent_mail', 'all_documents', 'contacts', 'deleted_items', 'discussion_threads', 'inbox', 'notes_inbox', 'sent_items', 'sent', 'straw', '2000_conference', 'active_international', 'avaya', 'bmc', 'bridge', 'bristol_babcock', 'colleen_koenig', 'compaq', 'computer_associates', 'continental_airlines', 'cooper_cameron', 'corestaff', 'dell', 'ebs', 'ees', 'enron_europe', 'etol', 'fedex', 'ge', 'hp', 'human_resources', 'kinko_s', 'nepco_europe', 'nepco', 'oec', 'pcc_values', 'personal', 'purchasing', 'requisite', 'sap', 'sarah_joy_hunter', 'sonoco', 'sony', 'sparefinders_com', 'tasks', 'universal_studios', 'vulcan_signs', 'weekly_report', 'drafts', 'funny', 'presentations', 'savedmail', 'vanderbilt', 'cal_articles', 'california', 'capx', 'dj_articles', 'memo_s', 'move', 'press_releases', 'var', 'calendar', 'entex', 'ffl', '1.', 'ect_admin', 'enron_power', 'power', 'so2___nox', 'to_do', 'ubsw_energy', '2001_plan', 'aec', 'analyst_assoc_program', 

In [5]:
print('Avg uniq directories ', df.groupby(['dir_owner'], as_index=False).agg({'dir': 'nunique'}).dir.mean())
print('Median uniq directories ', df.groupby(['dir_owner'], as_index=False).agg({'dir': 'nunique'}).dir.median())
df.groupby(['dir_owner'], as_index=False).agg({'dir': 'nunique'}).sort_values('dir', ascending=False)

Avg uniq directories  18.706666666666667
Median uniq directories  11.0


,dir_owner,dir
56,kean-s,187
7,beck-s,114
114,shackleton-s,111
116,shapiro-r,110
138,watson-k,69
...,...,...
81,meyers-a,3
143,whitt-m,3
119,slinger-r,3
44,harris-s,2


In [6]:
print(df['message'][0])

Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 


In [7]:
print(df['message'][2])

Message-ID: <24216240.1075855687451.JavaMail.evans@thyme>
Date: Wed, 18 Oct 2000 03:00:00 -0700 (PDT)
From: phillip.allen@enron.com
To: leah.arsdall@enron.com
Subject: Re: test
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Leah Van Arsdall
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf

test successful.  way to go!!!


In [ ]:

def quick_parse(email_text):
    msg = Parser(policy=policy.default).parsestr(email_text)
    return {
        'date': msg.get('Date'),
        'from': msg.get('From'),
        'to': msg.get('To'),
        'subject': msg.get('Subject')
    }

# Применяем к датафрейму
df['parsed'] = df['message'].apply(quick_parse)
df = pd.concat([df, df['parsed'].apply(pd.Series)], axis=1)
df

In [ ]:
nlp = spacy.blank("en")